<h1><center>Translate COCO</center></h1>


<!-- <center>Author: </center> -->

<b>github:</b> [ml_project_image_cap](https://github.com/Farhana100/ml_project_image_cap)

<!-- <b>paper:</b> [TIST: Transcriptome and Histopathological Image Integrative Analysis for Spatial Transcriptomics](https://doi.org/10.1016/j.gpb.2022.11.012) -->

[![Open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/19L0CG9cCwB4CjwnBc-y83AsXvWmfFKY9?usp=sharing)

### dependency installation

In [ ]:
!pip install googletrans==3.1.0a0 # specific version, or else gives error

### data download
source: [cocodataset](https://cocodataset.org/#download)

#### 2017 train/val

already saved in drive. get from there

##### image

In [ ]:
# !wget "http://images.cocodataset.org/zips/train2017.zip"

In [ ]:
# !wget "http://images.cocodataset.org/zips/val2017.zip"

##### captions

In [ ]:
# !wget "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

#### copy from drive

In [ ]:
#@markdown <center><h3>mount drive</h3></center>
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
!rsync -ah --progress "/content/drive/MyDrive/My Drive (1705092)/Study Materials/Level 4 - Term 2/CSE471-472/CSE472/ML project/data/coco-dataset-2017/annotations" "/content/"

In [ ]:
#@markdown <center><h3>unmount drive</h3></center>
drive.flush_and_unmount()

### import

In [ ]:
from googletrans import Translator as tr
import json
from tqdm import tqdm
from pathlib import Path
import argparse

### function def

In [ ]:
def refine_json(filepath):
    filepath = Path(filepath)
    with open(filepath, 'r', encoding='utf-8') as f:
        data_raw = json.load(f)

    images = data_raw['images']
    annotations = data_raw['annotations']


    records = dict()
    for img in images:
        record = dict()

        record['id'] = img['id']
        record['file_name'] = img['file_name']
        record['captions'] = list()

        records[img['id']] = record

    
    for ann in annotations:
        records[ann['image_id']]['captions'].append(ann['caption'])


    # json dump records
    refined_filepath = f'{filepath.parent}/{filepath.stem}_refined.json'
    with open(refined_filepath, 'w', encoding='utf-8') as f:
        json.dump(records, f, indent=4)
    
    return refined_filepath

In [ ]:
def get_records(refined_path):
    refined_path = Path(refined_path)

    data_ref = None
    with open(refined_path, 'r', encoding='utf-8') as f:
        data_ref = json.load(f)

    if data_ref is None:
        raise Exception('No data found in refined file')

    return data_ref if data_ref else []

In [ ]:
def translate(text, src='en', dest='bn'):
    translator = tr()
    result = translator.translate(text, src=src, dest=dest)

    result = [r.text for r in result]
    return result

In [ ]:
def save_temp(json_obj, save_path, count):
    save_path = Path(save_path)
    save_path = f'{save_path.parent}/{save_path.stem}_translated_{count}.json'
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(json_obj, f, indent=4)

### main

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--refine', dest='refine', default=False, action='store_true')
parser.add_argument('--translate', dest='translate', default=False, action='store_true')
parser.add_argument(dest='path', type=str, help='input json path')
parser.add_argument('--start_from', dest='start_from', default=0, type=int, help='starting index of key')
parser.add_argument('--save_path', dest='save_path', default='.', type=str, help='output json path')

In [ ]:
args =  parser.parse_args(args=['--refine', '--translate', '/content/annotations/captions_train2017.json', '--save_path', '/content/drive/MyDrive/My Drive (1705092)/Study Materials/Level 4 - Term 2/CSE471-472/CSE472/ML project/data/coco-dataset-2017/annotations/captions_train2017.json'])

In [ ]:
if args.refine:
    print('refining...')
    raw_path = args.path
    refined_path = refine_json(filepath=raw_path)
    print('done')

if args.translate:
    if args.refine:
        # given path was raw
        refined_path = refined_path
    else:
        # given path was refined
        refined_path = args.path
    
    records = get_records(refined_path=refined_path)
    translated_records = records.copy()
    records_keys = list(records.keys())
    records_keys = records_keys[args.start_from:]

    print('translating...')
    done_count = args.start_from
    for id in tqdm(records_keys):
        caps = translated_records[id]['captions']
        new_caps = translate(caps)
        translated_records[id]['captions'] = new_caps
        done_count += 1
        if done_count % 500 == 0:
            print('saving temp...')
            save_temp(json_obj=translated_records, save_path=args.save_path, count=done_count)
    print('done')

    if args.save_path is None or args.save_path == "." or args.save_path == "" or args.save_path == " ":
        save_path = f'{Path(refined_path).parent}/{Path(refined_path).stem}_translated.json'
    else:
        save_path = args.save_path

    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(translated_records, f, indent=4)